# Load dataset from kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json # 6 for owner, 0 for group and others

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Dataset URL: https://www.kaggle.com/datasets/alvarole/hand-and-face-detection-focused-on-sign-language
License(s): apache-2.0
^C


In [5]:
!pip install kagglehub

In [7]:
import kagglehub

path = kagglehub.dataset_download("alvarole/hand-and-face-detection-focused-on-sign-language")

print("Path to dataset files:", path)

Resuming download from 230686720 bytes (25390877758 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/alvarole/hand-and-face-detection-focused-on-sign-language?dataset_version_number=1 (230686720/25621564478) bytes left.


100%|██████████| 23.9G/23.9G [03:58<00:00, 107MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1


In [11]:
import os
import random
import shutil
from pathlib import Path

# === CONFIG ===
ROOT_DIR = Path("/root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1/hand_face_detection_dataset")
SUBSET_DIR = Path("/content/subset")

# How many images you want to keep per split (edit these numbers as you like)
TARGET_COUNTS = {
    "train": 20000,       # e.g. 50k train images
    "validation": 1000,  # e.g. 10k val images
    "test": 50000         # e.g. 10k test images
}

# If True, will DELETE the original "images" and "labels_yolo" folders after creating subset
DELETE_ORIGINAL = False  # <-- CHANGE TO True ONLY AFTER YOU CHECK THE SUBSET

# Random seed for reproducibility
random.seed(42)

# === HELPER FUNCTIONS ===

def list_image_files(folder):
    exts = {".jpg", ".jpeg", ".png", ".bmp"}
    return [f for f in folder.iterdir() if f.suffix.lower() in exts]

def ensure_dir(path: Path):
    path.mkdir(parents=True, exist_ok=True)

# === MAIN LOGIC ===

splits = ["train", "validation", "test"]

for split in splits:
    print(f"\n=== Processing split: {split} ===")

    img_dir = ROOT_DIR / "images" / split
    lbl_dir = ROOT_DIR / "labels_yolo" / split

    if not img_dir.exists():
        print(f"  [WARNING] Images folder does not exist: {img_dir}, skipping this split.")
        continue

    if not lbl_dir.exists():
        print(f"  [WARNING] Labels folder does not exist: {lbl_dir}, skipping this split.")
        continue

    # List all images in this split
    img_files = list_image_files(img_dir)
    num_images = len(img_files)
    print(f"  Found {num_images} images in {img_dir}")

    if num_images == 0:
        print("  No images found, skipping.")
        continue

    # Decide how many images to sample for this split
    target = min(TARGET_COUNTS.get(split, num_images), num_images)
    print(f"  Sampling {target} images for subset.")

    sampled_imgs = random.sample(img_files, target)

    # Create subset folders
    subset_img_dir = SUBSET_DIR / "images" / split
    subset_lbl_dir = SUBSET_DIR / "labels_yolo" / split
    ensure_dir(subset_img_dir)
    ensure_dir(subset_lbl_dir)

    # Copy images + matching YOLO label files
    for img_path in sampled_imgs:
        # Copy image
        dst_img_path = subset_img_dir / img_path.name
        shutil.copy2(img_path, dst_img_path)

        # Corresponding YOLO label (.txt)
        label_name = img_path.stem + ".txt"
        src_label_path = lbl_dir / label_name
        dst_label_path = subset_lbl_dir / label_name

        if src_label_path.exists():
            shutil.copy2(src_label_path, dst_label_path)
        else:
            # If there is no label file, you can either:
            #  - create an empty .txt
            #  - or skip it. Here we create an empty file.
            dst_label_path.touch()
            # print(f"  [INFO] No label for {img_path.name}, created empty label.")

    print(f"  Done split {split}. Subset images in: {subset_img_dir}")
    print(f"  Subset labels in: {subset_lbl_dir}")

print("\n✅ Subset creation finished.")
print(f"Subset root folder: {SUBSET_DIR}")

# === OPTIONAL: DELETE ORIGINAL LARGE FOLDERS ===
if DELETE_ORIGINAL:
    print("\n⚠️ DELETE_ORIGINAL = True → Deleting original 'images' and 'labels_yolo' folders...")
    orig_images = ROOT_DIR / "images"
    orig_labels = ROOT_DIR / "labels_yolo"

    if orig_images.exists():
        shutil.rmtree(orig_images)
        print(f"  Deleted: {orig_images}")

    if orig_labels.exists():
        shutil.rmtree(orig_labels)
        print(f"  Deleted: {orig_labels}")

    print("🗑️ Original dataset folders deleted.")
else:
    print("\nℹ️ DELETE_ORIGINAL is False. Original dataset is kept.")
    print("   After you verify the subset, you can set DELETE_ORIGINAL = True and re-run the last block.")



=== Processing split: train ===
  Found 369053 images in /root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1/hand_face_detection_dataset/images/train
  Sampling 20000 images for subset.
  Done split train. Subset images in: /content/subset/images/train
  Subset labels in: /content/subset/labels_yolo/train

=== Processing split: validation ===
  Found 59386 images in /root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1/hand_face_detection_dataset/images/validation
  Sampling 1000 images for subset.
  Done split validation. Subset images in: /content/subset/images/validation
  Subset labels in: /content/subset/labels_yolo/validation

=== Processing split: test ===
  Found 49041 images in /root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1/hand_face_detection_dataset/images/test
  Sampling 49041 images for subset.
  Done split test. Subset imag

In [12]:
# If True, will DELETE the original "images" and "labels_yolo" folders after creating subset
DELETE_ORIGINAL = True  # <-- CHANGE TO True ONLY AFTER YOU CHECK THE SUBSET

# Random seed for reproducibility
random.seed(42)

# === OPTIONAL: DELETE ORIGINAL LARGE FOLDERS ===
if DELETE_ORIGINAL:
    print("\n⚠️ DELETE_ORIGINAL = True → Deleting original 'images' and 'labels_yolo' folders...")
    orig_images = ROOT_DIR / "images"
    orig_labels = ROOT_DIR / "labels_yolo"

    if orig_images.exists():
        shutil.rmtree(orig_images)
        print(f"  Deleted: {orig_images}")

    if orig_labels.exists():
        shutil.rmtree(orig_labels)
        print(f"  Deleted: {orig_labels}")

    print("🗑️ Original dataset folders deleted.")
else:
    print("\nℹ️ DELETE_ORIGINAL is False. Original dataset is kept.")
    print("   After you verify the subset, you can set DELETE_ORIGINAL = True and re-run the last block.")



⚠️ DELETE_ORIGINAL = True → Deleting original 'images' and 'labels_yolo' folders...
  Deleted: /root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1/hand_face_detection_dataset/images
  Deleted: /root/.cache/kagglehub/datasets/alvarole/hand-and-face-detection-focused-on-sign-language/versions/1/hand_face_detection_dataset/labels_yolo
🗑️ Original dataset folders deleted.


In [ ]:
import os
from pathlib import Path

ROOT = Path("")

splits = ["train", "val", "test"]

for split in splits:
    label_dir = ROOT / "labels" / split
    img_dir = ROOT / "images" / split

    print(f"\n=== Processing split: {split} ===")

    for label_file in label_dir.glob("*.txt"):
        new_lines = []

        with open(label_file, "r") as f:
            for line in f:
                parts = line.strip().split()

                if len(parts) < 5:
                    continue

                cls = parts[0]

                if cls == "0":
                    # remove face label
                    continue

                if cls == "1":
                    # hand label -> change to 0
                    parts[0] = "0"
                    new_lines.append(" ".join(parts))

        # Save new label file (only hands)
        with open(label_file, "w") as f:
            for line in new_lines:
                f.write(line + "\n")

        # OPTIONAL: delete image if no hand left
        if len(new_lines) == 0:
            img_path = img_dir / (label_file.stem + ".jpg")
            if img_path.exists():
                img_path.unlink()   # delete image
            label_file.unlink()      # delete empty label file

    print(f"Done processing {split}.")